In [41]:
import pandas as pd
import pycountry

#### Working with data set ds.salaries.csv, with goal of converting ISO 3166 country code to country name column

In [42]:
df = pd.read_csv("ds_salaries.csv", index_col="#")
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
#,,,,,,,,,,,
0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L


#### Testing conversion with pycountry

In [43]:
df.loc[1]["company_location"]
print(pycountry.countries.get(alpha_2=df.loc[1]["company_location"]).name)

Japan


#### Issue: Certain country codes causing LookupError.  New goal to identify erronous country code.

In [44]:
df.sort_values("company_location", inplace=True)
# print(df["company_location"].unique())
for country in df["company_location"].unique():
    try:

        print(f"{country} country name is {pycountry.countries.get(alpha_2=country).name}")
    except LookupError:
        print(f"{country} country name is AN INVALID FORMAT")


AE country name is United Arab Emirates
AS country name is American Samoa
AT country name is Austria
AU country name is Australia
BE country name is Belgium
BR country name is Brazil
CA country name is Canada
CH country name is Switzerland
CL country name is Chile
CN country name is China
CO country name is Colombia
CZ country name is Czechia
DE country name is Germany
DK country name is Denmark
DZ country name is Algeria
EE country name is Estonia
ES country name is Spain
FR country name is France
GB country name is United Kingdom
GR country name is Greece
HN country name is Honduras
HR country name is Croatia
HU country name is Hungary
IE country name is Ireland
IL country name is Israel
IN country name is India
IQ country name is Iraq
IR country name is Iran, Islamic Republic of
IT country name is Italy
JP country name is Japan
KE country name is Kenya
LU country name is Luxembourg
MD country name is Moldova, Republic of
MT country name is Malta
MX country name is Mexico
MY country 

#### Below attempt is still throwing lookup error - switching to using extra .csv file of countries

In [45]:
try:
    df["company_location"] = (pycountry.countries.lookup(df["company_location"]).name)
    df.head()
except LookupError:
    print("Lookup Error")

Lookup Error


#### Importing of country name/code csv, and removal of unnecessary columns (preparation for merging DF's)

In [46]:
country_df = pd.read_csv("wikipedia-iso-country-codes.csv", index_col="Alpha-2")
country_df.drop(["Alpha-3", "numeric_code", "ISO_3166-2"], axis=1, inplace=True)
country_df.head(70)


,country_name
Alpha-2,
AF,Afghanistan
AX,Åland Islands
AL,Albania
DZ,Algeria
AS,American Samoa
...,...
GQ,Equatorial Guinea
ER,Eritrea
EE,Estonia


#### Merging of DF's based on "company_location" country code

In [47]:
#print(country_df.loc[df.loc[1]["company_location"]]["country_name"])
merged_df = pd.merge(df, country_df, left_on="company_location", right_on="Alpha-2")
print(merged_df["country_name"].unique())


['United Arab Emirates' 'American Samoa' 'Austria' 'Australia' 'Belgium'
 'Brazil' 'Canada' 'Switzerland' 'Chile' 'China' 'Colombia'
 'Czech Republic' 'Germany' 'Denmark' 'Algeria' 'Estonia' 'Spain' 'France'
 'United Kingdom' 'Greece' 'Honduras' 'Croatia' 'Hungary' 'Ireland'
 'Israel' 'India' 'Iraq' 'Iran, Islamic Republic of' 'Italy' 'Japan'
 'Kenya' 'Luxembourg' 'Moldova, Republic of' 'Malta' 'Mexico' 'Malaysia'
 'Nigeria' 'Netherlands' 'New Zealand' 'Pakistan' 'Poland' 'Portugal'
 'Romania' 'Russian Federation' 'Singapore' 'Slovenia' 'Turkey' 'Ukraine'
 'United States' 'Viet Nam']


#### Cleanup of columns including:
* Dropping company_location column
* Renaming country_name to company_location

In [48]:
merged_df_two = merged_df.drop("company_location", axis=1).rename(columns={"country_name": "company_location"})
merged_df_two.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_size,company_location
0,2020,MI,FT,Lead Data Scientist,115000,USD,115000,AE,0,L,United Arab Emirates
1,2022,SE,FT,Machine Learning Engineer,65000,USD,65000,AE,100,S,United Arab Emirates
2,2022,SE,FT,Machine Learning Engineer,120000,USD,120000,AE,100,S,United Arab Emirates
3,2021,EN,FT,AI Scientist,1335000,INR,18053,IN,100,S,American Samoa
4,2020,MI,FT,Data Engineer,65000,EUR,74130,AT,50,L,Austria


#### Rinse & repeat with employee_residence column

In [49]:
employee_merged_df = pd.merge(merged_df_two, country_df, left_on="employee_residence", right_on="Alpha-2")
employee_merged_df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_size,company_location,country_name
0,2020,MI,FT,Lead Data Scientist,115000,USD,115000,AE,0,L,United Arab Emirates,United Arab Emirates
1,2022,SE,FT,Machine Learning Engineer,65000,USD,65000,AE,100,S,United Arab Emirates,United Arab Emirates
2,2022,SE,FT,Machine Learning Engineer,120000,USD,120000,AE,100,S,United Arab Emirates,United Arab Emirates
3,2021,EN,FT,AI Scientist,1335000,INR,18053,IN,100,S,American Samoa,India
4,2021,EN,FT,Big Data Engineer,435000,INR,5882,IN,0,L,Switzerland,India


In [50]:
final_df = employee_merged_df.drop("employee_residence", axis=1).rename(columns={"country_name": "employee_residence"})
final_df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,remote_ratio,company_size,company_location,employee_residence
0,2020,MI,FT,Lead Data Scientist,115000,USD,115000,0,L,United Arab Emirates,United Arab Emirates
1,2022,SE,FT,Machine Learning Engineer,65000,USD,65000,100,S,United Arab Emirates,United Arab Emirates
2,2022,SE,FT,Machine Learning Engineer,120000,USD,120000,100,S,United Arab Emirates,United Arab Emirates
3,2021,EN,FT,AI Scientist,1335000,INR,18053,100,S,American Samoa,India
4,2021,EN,FT,Big Data Engineer,435000,INR,5882,0,L,Switzerland,India


#### Final check: Comparison of country names in cost of living data set to ensure all country names match

In [54]:
print(final_df["employee_residence"].unique())


['United Arab Emirates' 'India' 'Austria' 'Germany' 'Australia' 'Belgium'
 'Brazil' 'Canada' 'United Kingdom' 'United States' 'Switzerland' 'Chile'
 'China' 'Jersey' 'Colombia' 'Tunisia' 'Czech Republic' 'Serbia'
 'Pakistan' 'Netherlands' 'France' 'Greece' 'Denmark' 'Algeria' 'Estonia'
 'Spain' 'Poland' 'Romania' 'Viet Nam' 'Hong Kong' 'Honduras' 'Croatia'
 'Hungary' 'Ireland' 'Singapore' 'Iraq' 'Iran, Islamic Republic of'
 'Italy' 'Japan' 'Kenya' 'Luxembourg' 'Portugal' 'Moldova, Republic of'
 'Malta' 'Argentina' 'Mexico' 'Nigeria' 'New Zealand' 'Russian Federation'
 'Slovenia' 'Turkey' 'Ukraine' 'Bolivia, Plurinational State of'
 'Malaysia' 'Philippines' 'Bulgaria' 'Puerto Rico']


In [52]:
living_df = pd.read_csv("Cost_of_Living_Index_2022.csv")
living_df["Country"].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Argentina', 'Armenia',
       'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain',
       'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize',
       'Bermuda', 'Bolivia', 'Bosnia And Herzegovina', 'Botswana',
       'Brazil', 'Bulgaria', 'Cambodia', 'Cameroon', 'Canada', 'Chile',
       'China', 'Colombia', 'Costa Rica', 'Croatia', 'Cuba', 'Cyprus',
       'Czech Republic', 'Denmark', 'Dominican Republic', 'Ecuador',
       'Egypt', 'El Salvador', 'Estonia', 'Ethiopia', 'Fiji', 'Finland',
       'France', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Guatemala',
       'Guernsey', 'Honduras', 'Hong Kong', 'Hungary', 'Iceland', 'India',
       'Indonesia', 'Iran', 'Iraq', 'Ireland', 'Israel', 'Italy',
       'Ivory Coast', 'Jamaica', 'Japan', 'Jersey', 'Jordan',
       'Kazakhstan', 'Kenya', 'Kosovo (Disputed Territory)', 'Kuwait',
       'Latvia', 'Lebanon', 'Libya', 'Lithuania', 'Luxembourg', 'Macao',
       'Malaysia', 'Maldives',

#### Outliers to fix in final_df:
* Viet Nam --> Vietnam
* Moldova, Republic of --> Moldova
* Russian Federation --> Russia
* Iran, Islamic Republic of --> Iran
* Bolivia, Plurinational State of --> Bolivia
* American Samoa --> No matching field - drop records to avoid student confusion


#### Employee residence column cleanup

In [57]:
final_df["employee_residence"] = final_df["employee_residence"].replace("Viet Nam", "Vietnam")
final_df["employee_residence"] = final_df["employee_residence"].replace("Moldova, Republic of", "Moldova")
final_df["employee_residence"] = final_df["employee_residence"].replace("Iran, Islamic Republic of", "Iran")
final_df["employee_residence"] = final_df["employee_residence"].replace("Bolivia, Plurinational State of", "Bolivia")
final_df["employee_residence"] = final_df["employee_residence"].replace("Russian Federation", "Russia")
print(final_df["employee_residence"].unique())

['United Arab Emirates' 'India' 'Austria' 'Germany' 'Australia' 'Belgium'
 'Brazil' 'Canada' 'United Kingdom' 'United States' 'Switzerland' 'Chile'
 'China' 'Jersey' 'Colombia' 'Tunisia' 'Czech Republic' 'Serbia'
 'Pakistan' 'Netherlands' 'France' 'Greece' 'Denmark' 'Algeria' 'Estonia'
 'Spain' 'Poland' 'Romania' 'Vietnam' 'Hong Kong' 'Honduras' 'Croatia'
 'Hungary' 'Ireland' 'Singapore' 'Iraq' 'Iran' 'Italy' 'Japan' 'Kenya'
 'Luxembourg' 'Portugal' 'Moldova' 'Malta' 'Argentina' 'Mexico' 'Nigeria'
 'New Zealand' 'Russia' 'Slovenia' 'Turkey' 'Ukraine' 'Bolivia' 'Malaysia'
 'Philippines' 'Bulgaria' 'Puerto Rico']


#### Company Location Column Cleanup

In [58]:
final_df["company_location"] = final_df["company_location"].replace("Viet Nam", "Vietnam")
final_df["company_location"] = final_df["company_location"].replace("Moldova, Republic of", "Moldova")
final_df["company_location"] = final_df["company_location"].replace("Iran, Islamic Republic of", "Iran")
final_df["company_location"] = final_df["company_location"].replace("Bolivia, Plurinational State of", "Bolivia")
final_df["company_location"] = final_df["company_location"].replace("Russian Federation", "Russia")
print(final_df["company_location"].unique())

['United Arab Emirates' 'American Samoa' 'Switzerland' 'India'
 'United States' 'Austria' 'Germany' 'Australia' 'Belgium' 'Brazil'
 'Canada' 'United Kingdom' 'France' 'Chile' 'China' 'Colombia'
 'Czech Republic' 'Pakistan' 'Netherlands' 'Spain' 'Luxembourg' 'Denmark'
 'Greece' 'Algeria' 'Estonia' 'Romania' 'Poland' 'Vietnam' 'Honduras'
 'Croatia' 'Hungary' 'Ireland' 'Israel' 'Singapore' 'Iraq' 'Iran' 'Italy'
 'Japan' 'Malaysia' 'Kenya' 'Portugal' 'Moldova' 'Malta' 'Mexico'
 'Nigeria' 'New Zealand' 'Russia' 'Slovenia' 'Turkey' 'Ukraine']


#### Removal of American Samoa references from final_df

In [63]:
final_cleaned_df = final_df[final_df.company_location != "American Samoa"]
final_cleaned_df["company_location"].unique()

array(['United Arab Emirates', 'Switzerland', 'India', 'United States',
       'Austria', 'Germany', 'Australia', 'Belgium', 'Brazil', 'Canada',
       'United Kingdom', 'France', 'Chile', 'China', 'Colombia',
       'Czech Republic', 'Pakistan', 'Netherlands', 'Spain', 'Luxembourg',
       'Denmark', 'Greece', 'Algeria', 'Estonia', 'Romania', 'Poland',
       'Vietnam', 'Honduras', 'Croatia', 'Hungary', 'Ireland', 'Israel',
       'Singapore', 'Iraq', 'Iran', 'Italy', 'Japan', 'Malaysia', 'Kenya',
       'Portugal', 'Moldova', 'Malta', 'Mexico', 'Nigeria', 'New Zealand',
       'Russia', 'Slovenia', 'Turkey', 'Ukraine'], dtype=object)

In [67]:
final_cleaned_df.to_csv("Data_Scientist_Salaries.csv", index=False)